In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import smtplib
import time
import mail 
from dotenv import load_dotenv
import os

In [14]:
load_dotenv()

True

In [15]:
SENDER_EMAIL = os.getenv("SENDER_EMAIL")
SENDER_PASSWORD = os.getenv("SENDER_PASSWORD")
RECEIVER_EMAIL = os.getenv("RECEIVER_EMAIL")

In [16]:
def main_code(wait, previously_open_courses):
    currently_open_courses = set()
    
    course_title_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, COURSE_TITLE_SELECTOR)))
    class_code_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, CLASS_CODE_SELECTOR)))

    for title_element, code_element in zip(course_title_elements, class_code_elements):
        class_code = int(code_element.text.strip())
        if class_code not in NOT_INCLUDED:
            currently_open_courses.add(class_code)

    newly_opened_courses = currently_open_courses - previously_open_courses

    if newly_opened_courses:
        print(f"✅ New courses found: {newly_opened_courses}")
        mail.send_notification_email(newly_opened_courses, SENDER_EMAIL, SENDER_PASSWORD, RECEIVER_EMAIL)


    closed_courses = previously_open_courses - currently_open_courses
    return currently_open_courses


In [17]:
driver = webdriver.Chrome()
driver.get("https://prodweb.snu.in/psp/CSPROD/EMPLOYEE/HRMS/?cmd=login")
elem = driver.find_element(By.NAME, "userid")
elem.clear()
elem.send_keys("ss691")
elem2 = driver.find_element(By.NAME, "pwd")
elem2.clear()
elem2.send_keys("6W2PYK-AVVnc~s)")
elem2.send_keys(Keys.RETURN)

In [18]:
wait = WebDriverWait(driver, 10)

In [19]:
wait.until(EC.presence_of_element_located((By.ID, "fldra_CO_EMPLOYEE_SELF_SERVICE"))).click()
wait.until(EC.presence_of_element_located((By.ID, "fldra_HCCC_SS_CATALOG"))).click()
wait.until(EC.presence_of_element_located((By.ID, "crefli_HC_CLASS_SEARCH"))).click()

In [20]:
wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ptifrmtgtframe")))
SEARCH_TERM = 'Core Common Curriculum'
SEARCH_TERM_INPUT_ID = "SSR_CLSRCH_WRK_SUBJECT_SRCH$0"
SEARCH_BUTTON_ID = "CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH"
wait.until(EC.element_to_be_clickable((By.ID, SEARCH_TERM_INPUT_ID))).send_keys(SEARCH_TERM)
driver.find_element(By.ID, SEARCH_BUTTON_ID).click()
driver.find_element(By.ID, "SSR_CLSRCH_WRK_SUBJECT_SRCH$0").click()

In [21]:
NOT_INCLUDED = {1761, 1727, 1693, 1533, 1536, 1539, 1541, 1546, 1547, 1548, 1551, 1552, 1553, 1556, 1557, 1558, 1559, 1561, 1562, 1565, 1566, 1569, 1570, 2075, 2076}
COURSE_TITLE_SELECTOR = 'td.PAGROUPBOXLABELLEVEL1'
CLASS_CODE_SELECTOR = "a[id^='MTG_CLASS_NBR$']"

In [ ]:
known_open_courses = set()
known_open_courses = main_code(wait, known_open_courses)
time.sleep(10)
while True:
    wait.until(EC.presence_of_element_located((By.ID, "CLASS_SRCH_WRK2_SSR_PB_MODIFY"))).click()
    wait.until(EC.presence_of_element_located((By.ID, "CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH"))).click()
    time.sleep(60)
    known_open_courses = main_code(wait, known_open_courses)

✅ New courses found: {2306}
✅ Successfully sent email notification.
